# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [14]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [15]:
!cat talks.tsv

category	date	title	venue	location	talk_url	venue_url	description
Talk	2020-08-30	New integer and Bilevel Formulations for the k-Vertex Cut Problem	22ème séminaire du groupe POC (online)		https://www.youtube.com/watch?v=v66IrAejl7k&feature=youtu.be	https://www.lamsade.dauphine.fr/poc/?q=node/72	
Talk	2020-10-12	Branch-and-Cut Solvers for Mixed-Integer Bilevel Linear Programs	Autumn School on Bilevel Optimization	Uni Trier, Germany	https://alop.uni-trier.de/wp-content/uploads/2020/10/ALOP_Ljubić.pdf	https://alop.uni-trier.de/event/autumn-school-on-bilevel-optimization/	
Talk	2020-01-20	Branch-and-Benders-cut algorithms: modern implementations of Benders Decomposition	9th Winter School on Network Optimization	Estoril, Portugal	http://netopt2020.campus.ciencias.ulisboa.pt/sites/netopt2020/files/Abs_Ivana.pdf	http://netopt2020.campus.ciencias.ulisboa.pt/	
Talk	2018-08-23	A new branch-and-cut algorithm for interdiction problems with monotonicity	USC Center for Artificial Intelligence in Soc

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [16]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,category,date,title,venue,location,talk_url,venue_url,description
0,Talk,2020-08-30,New integer and Bilevel Formulations for the k...,22ème séminaire du groupe POC (online),NaN,https://www.youtube.com/watch?v=v66IrAejl7k&fe...,https://www.lamsade.dauphine.fr/poc/?q=node/72,NaN
1,Talk,2020-10-12,Branch-and-Cut Solvers for Mixed-Integer Bilev...,Autumn School on Bilevel Optimization,"Uni Trier, Germany",https://alop.uni-trier.de/wp-content/uploads/2...,https://alop.uni-trier.de/event/autumn-school-...,NaN
2,Talk,2020-01-20,Branch-and-Benders-cut algorithms: modern impl...,9th Winter School on Network Optimization,"Estoril, Portugal",http://netopt2020.campus.ciencias.ulisboa.pt/s...,http://netopt2020.campus.ciencias.ulisboa.pt/,NaN
3,Talk,2018-08-23,A new branch-and-cut algorithm for interdictio...,USC Center for Artificial Intelligence in Society,USA,NaN,https://www.cais.usc.edu/,NaN
4,Talk,2018-10-08,Very Large Scale Covering Location Problems in...,Workshop on Smart Cities Analytics,"Ivey Business School, Canada",http://www.iveysmartcities.tk/slides/Ljubić_Sm...,http://www.iveysmartcities.tk/index.html#speakers,NaN
5,Talk,2018-03-26,Exact General-Purpose Solvers for Mixed-Intege...,37ème Journées Franciliennes de Recherche Opér...,"CNAM, Paris, France",slides/JFRO_Ljubić.pdf,http://www.lamsade.dauphine.fr/~jfro/JourneesP...,NaN
6,Talk,2018-08-17,Very Large Scale Covering Location Problems in...,GERAD Seminar,"Montreal, Canada",NaN,https://www.gerad.ca/en/events/1608,NaN
7,Talk,2018-06-11,New Branch-and-Cut Algorithms for Mixed-Intege...,Séminaire Parisien d'Optimisation,"Institut Henri Poincaré, Paris, France",slides/Workshop_Ljubić.pdf,NaN,NaN
8,Invited Speaker,2017-06-16,A dual-ascent-based branch-and-bound framework...,Journées Polyèdres et Optimisation Combinatoir...,"University Paris 13, France",NaN,http://www.lamsade.dauphine.fr/~poc/jpoc10/pro...,NaN
9,Talk,2016-10-08,A new general-purpose algorithm for mixed-inte...,15ème séminaire du groupe POC - Combinatorial ...,"Université Pierre et Marie Curie, Paris",NaN,http://www.lamsade.dauphine.fr/~poc/poc/?q=nod...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [17]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [18]:
loc_dict = {}

for row, item in talks.iterrows():
    url_slug = '-'.join([i for i in item.title.lower().split()])
    
    md_filename = str(item.date) + "-" + url_slug + ".md"
    html_filename = str(item.date) + "-" + url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.category)) > 3:
        md += 'category: "' + item.category + '"\n'
    else:
        md += 'category: "Talk"\n'
    
    md += 'type: "othertalk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    if len(str(item.venue)) > 3:
        if len(str(item.venue_url)) > 3:
            md += "\nThis talk was part of [" + item.venue + "](" + item.venue_url + ")" 
        else:
            md += "\nThis talk was part of " + item.venue

        if len(str(item.talk_url)) > 3:
            if 'slides' in str(item.talk_url):
                md += " and more information can be found [here]({{site.url}}/docs/" + item.talk_url + ")\n" 
            else:
                md += " and more information can be found [here](" + item.talk_url + ")\n" 
        else:
            md += "."
    else:
        if len(str(item.talk_url)) > 3:
            md += "\nMore information can be found [here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [13]:
!ls ../_talks

2006-07-31-optimization-in-telecommunication-networks:-multicommodity-flow,-facility-location-and-steiner-tree-problems.md
2007-03-19-connected-facility-location-in-the-design-of-telecommunication-networks.md
2010-06-27-two-stage-branch-&-cut-for-two-stage-stochastic-network-design-problems.md
2010-08-16-dissaggregated-flow-formulation-for-the-single-source-network-loading-problem.md
2010-08-16-opttelnets:-algorithmische-ansätze.md
2011-10-21-the-generalized-regenerator-location-problem.md
2012-01-27-solving-two-stage-stochastic-steiner-tree-problems-by-two-stage-branch-and-cut.md
2012-04-04-optimization-tools-for-last-mile-access-networks.md
2012-04-26-the-recoverable-robust-two-level-network-design-problem.md
2012-07-04-solving-2-stage-stochastic-network-design-problems-by-2-stage-branch-and-cut.md
2012-11-22-the-maximum-weight-connected-subgraph-problem.md
2013-08-29-the-maximum-weight-connected-subgraph-problem:-applications-in-bioinformatics.md
2014-05-13-the-recoverable-robust-fa

In [15]:
!cat ../_talks/2012-03-01-mixed-integer-bilevel-optimization.md

---
title: "Mixed Integer Bilevel Optimization"
collection: talks
type: "Talk"
permalink: /talks/2012-03-01-mixed-integer-bilevel-optimization
venue: "SIAM Conference on Optimization (OP21)"
---
This talk was part of [SIAM Conference on Optimization (OP21)](https://www.siam.org/conferences/cm/program/invited-presentations/op21-invited-presentations) and more information can be found [here](https://meetings.siam.org/sess/dsp_programsess.cfm?SESSIONCODE=72554)
